In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set();

In [ ]:
df= pd.read_csv('../input/bank-marketing-dataset/bank.csv')

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.describe().T

**Полное описание**: 
https://www.researchgate.net/figure/Bank-marketing-data-set-descriptions_tbl1_283761844
Пояснение некоторых признаков:
poutcome: результат предыдущей маркетинговой кампании (категориальный: «неудача», «несуществующий», «успех»)
previous: количество контактов, выполненных до этой кампании для этого клиента (числовое значение)
campaign: количество контактов, выполненных во время этой кампании для этого клиента (числовое, включая последний контакт)

Целевая переменная: 
duration: клиент подписался на срочный вклад? (бинарный: «yes», «no»)


Задача классификации: вероятность, что данный клиент может взять срочный депозит

In [ ]:
df['deposit'].value_counts(normalize=True)

In [ ]:
# df['deposit'].hist()

In [ ]:
df['deposit'].value_counts().plot(kind='bar', label='deposit')
plt.ylabel('count')
plt.xlabel('deposit')

Как видим, в выборке почти половина людей согласилась на срочный депозит, то есть классы сбалансированы.

In [ ]:
df['education'].value_counts(normalize=True)

In [ ]:
df['job'].value_counts(normalize=True)

In [ ]:
df['poutcome'].value_counts(normalize=True)

In [ ]:
sns.boxplot(df['balance'])

In [ ]:
df['balance'].hist(bins=20);

In [ ]:
sns.boxplot(df['duration'])

In [ ]:
df['duration'].hist(bins=30);

В выборке есть незначительное количество людей с отрицательным баллансом счёта.

In [ ]:
def outliers_indices(feature):
    mid = df[feature].mean()
    sigma = df[feature].std()
    return df[(df[feature] < mid - 3*sigma) | (df[feature] > mid + 3*sigma)].index

In [ ]:
wrong_dur=outliers_indices('duration')
wrong_bal=outliers_indices('balance')
out=set(wrong_bal|wrong_dur)
len(out)

Выбросов не очень много, для данного датасета их удаление не критично

In [ ]:
df['deposit'].value_counts(normalize=True)

In [ ]:
df['deposit'].hist()

Как видим, в выборке почти половина людей согласилась на срочный депозит

In [ ]:
df.drop(out, inplace=True)

In [ ]:
sns.pairplot(df);

In [ ]:
df['deposit']=df['deposit'].map({'no': 0,'yes': 1})
#Можно трактовать и как ранговый (значит, можно вычислять, например, коэфициент Спирмена) и как категориальный
# df['deposit']

In [ ]:
sns.heatmap(df.corr(method='spearman'));

Как видим, есть прямая зависимость между длительностью разговора и целевой переменной. Также можно искать между целевой переменной и баллансом клиента. Можно также рассмотреть зависимость от количества времени, прошедшего с последнего вызова и количеством предыдущих контактов, поскольку между ними тест также показывает зависимость.

In [ ]:
plt.figure(figsize=(25, 250))
sns.countplot(y='duration', hue='deposit', data=df);

In [ ]:
df.groupby('deposit')['duration'].mean()

In [ ]:
df.groupby('deposit')['balance'].mean().plot(kind='bar')
plt.ylabel('balance')

In [ ]:
df.groupby('deposit')['balance'].mean()

In [ ]:
from scipy.stats import pointbiserialr
pointbiserialr(df['balance'],df['deposit'])

In [ ]:
pointbiserialr(df['duration'],df['deposit'])

Как видим, присутствует статистически значимая прямая слабая связь между размером баланса клиента и тем, согласился ли он на срочный депозит. В среднем, баланс тех, кто согласился оформить депозит выше на 400$.

Также есть ожидаемая достаточно сильная  статистически значимая прямая взаимосвять между продолжительностью разговора и взял ли клиент депозит.  Большинство людей, которые отказались, сделали это в первые минуты разговора. А с большинством людей, которые согласились, разговор длился больше 8-ми минут.

In [ ]:
sns.countplot(y='previous', hue='deposit', data=df[(df['previous']<5)]);

In [ ]:
plt.figure(figsize=(15, 15))
sns.countplot(y='previous', hue='deposit', data=df[(df['previous']>5)]);

In [ ]:
sns.countplot(y='pdays', hue='deposit', data=df[df['pdays']<0]);

In [ ]:
plt.figure(figsize=(15, 100))
sns.countplot(y='pdays', hue='deposit', data=df[df['pdays']>0]);

In [ ]:
# df[(df['deposit']==1)]['month'].value_counts()
df['deposit']

In [ ]:
pointbiserialr(df['previous'],df['deposit'])

In [ ]:
pointbiserialr(df['pdays'],df['deposit'])

Как видим, слабая значимая прямая взаимосвязь есть. То есть те, кто уже ложил депозит, скорее всего, положат его ещё раз. Наличие связи между положившим количеством людей и прошедшим периодом тоже есть, также прямая значимая, но слабая. Хотя это связано скорее с глобальными факторами, а не индивидуально относительно клиентов (например, акция в каком-то месяце в банке, или общее временное улучшение финансового положения граждан)

In [ ]:
df['duration']=df['duration']/60

In [ ]:
df['default']=df['default'].map({'no':0,'yes':1})
df['housing']=df['housing'].map({'no':0,'yes':1})
df['loan']=df['loan'].map({'no':0,'yes':1})
df.info()

In [ ]:
# dummy df
ddf = pd.get_dummies(df, columns=['job', 'education', 'marital', 'contact', 'poutcome', 'month'])
ddf.info()

In [ ]:
ddf.head().T

In [ ]:
from sklearn.model_selection import train_test_split
X=ddf.drop('deposit',axis=1)
y=ddf['deposit']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=12)

In [ ]:
bank_dataframe = pd.DataFrame(X_train, columns=X.columns)
grr = pd.plotting.scatter_matrix(bank_dataframe, 
                                 c=y_train, 
                                 figsize=(200, 200), 
                                 marker='o',
                                 hist_kwds={'bins': 20}, s=60, alpha=.8)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [ ]:
knn.fit(X_train,y_train)

In [ ]:
knn.score(X_valid, y_valid)

In [ ]:
y_pred = knn.predict(X_valid)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_valid, y_pred)

Данная модель работает верно с вероятностью 66%, что, скорее всего, является не самым лучшим результатом из возможных.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kf = KFold(n_splits=5, shuffle=True, random_state=42)
knn = KNeighborsClassifier(n_neighbors=5)
scores = cross_val_score(knn, X, y, 
                         cv=kf, scoring='accuracy')
# Поскольку итоговый результат однозначен, данная метрика качества вполне подходит
print(scores)
mean_score = scores.mean()
print(mean_score)

In [ ]:
from sklearn.model_selection import GridSearchCV
knn_params = {'n_neighbors': np.arange(1, 51),'metric':['minkowski','chebyshev']}
knn_grid = GridSearchCV(knn, 
                        knn_params, 
                        scoring='accuracy',
                        cv=kf)
knn_grid.fit(X_train, y_train)

In [ ]:
knn_grid.best_estimator_

In [ ]:
knn_grid.best_score_

In [ ]:
knn_grid.best_params_

In [ ]:
pd.DataFrame(knn_grid.cv_results_).T

In [ ]:
knn_grid.score(X_valid, y_valid)

In [ ]:
y_pred = knn_grid.predict(X_valid)
accuracy_score(y_valid, y_pred)

In [ ]:
best_knn = KNeighborsClassifier(n_neighbors=9)
y_pred = best_knn.fit(X_train, y_train).predict(X_valid)
accuracy_score(y_valid, y_pred)

In [ ]:
results_df = pd.DataFrame(knn_grid.cv_results_)
plt.plot(results_df[(results_df['param_metric']=='minkowski')]['param_n_neighbors'], results_df[(results_df['param_metric']=='minkowski')]['mean_test_score'])
plt.xlabel('n_neighbors')
plt.ylabel('Test accuracy')
plt.title('Validation curve')
plt.show()

**Рассмотрение другой метрики**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=9,weights='distance')
knn_params = {'p': np.linspace(1, 10,num=200)}
knn_grid = GridSearchCV(knn,
                        knn_params,
                        scoring='accuracy',
                        cv=kf)
knn_grid.fit(X_train, y_train)

In [ ]:
knn_grid.best_estimator_

In [ ]:
knn_grid.best_score_

In [ ]:
knn_grid.best_params_

In [ ]:
knn_grid.score(X_valid, y_valid)

In [ ]:
y_pred = knn_grid.predict(X_valid)
accuracy_score(y_valid, y_pred)

In [ ]:
best_knn = KNeighborsClassifier(n_neighbors=13,metric='manhattan',weights='distance')
#При манхэттенской метрике число соседей 13 более эффективно
y_pred = best_knn.fit(X_train, y_train).predict(X_valid)
accuracy_score(y_valid, y_pred)

In [ ]:
scores = cross_val_score(best_knn, X, y, 
                         cv=kf, scoring='accuracy')
scores.mean()

Итак, модель построенная на 13 соседях, манхэттенской метрике и с учётом дистанции между объектами работает с точностью почти 70%

In [ ]:
from sklearn.neighbors import RadiusNeighborsClassifier
rnc=RadiusNeighborsClassifier(weights='distance')
rnc_params = {'radius': np.linspace(500, 5000,num=50)}
rnc_grid = GridSearchCV(rnc,
                        rnc_params,
                        scoring='accuracy',
                        cv=kf)
rnc_grid.fit(X_train, y_train)

In [ ]:
rnc_grid.best_estimator_


In [ ]:
rnc_grid.best_score_


In [ ]:
rnc_grid.best_params_


In [ ]:
# rnc_grid.score(X_valid, y_valid)


Как видим, лучший радиус- наименьший, однако даже он даёт результат хуже метода knn. Брать радиус ещё меньших размеров не эффективно, так как в него не попадают объекты выборки.

In [ ]:
from sklearn.neighbors import RadiusNeighborsClassifier
rnc=RadiusNeighborsClassifier(weights='distance', radius=1000)
rnc_params = {'p': np.linspace(1, 10,num=10)}
rnc_grid = GridSearchCV(rnc,
                        rnc_params,
                        scoring='accuracy',
                        cv=kf)
rnc_grid.fit(X_train, y_train)

In [ ]:
rnc_grid.best_estimator_


In [ ]:
rnc_grid.best_score_


In [ ]:
rnc_grid.best_params_


В данном методе самой эффективной снова оказалась манхэттенская метрика

In [ ]:
pd.DataFrame(rnc_grid.cv_results_).T

In [ ]:
# rnc_grid.score(X_valid, y_valid)

Однако даже при увеличенном радиусе данный метод не даёт желаемых результатов

In [ ]:
from sklearn.neighbors import NearestCentroid
nc=NearestCentroid()
nc_params = {'metric':['manhattan','chebyshev']}
nc_grid = GridSearchCV(nc,
                        nc_params,
                        scoring='accuracy',
                        cv=kf)
nc_grid.fit(X_train, y_train)

In [ ]:
nc_grid.best_estimator_


In [ ]:
nc_grid.best_score_


In [ ]:
nc_grid.best_params_


In [ ]:
pd.DataFrame(nc_grid.cv_results_).T


In [ ]:
nc_grid.score(X_valid, y_valid)


In [ ]:
best_nc = NearestCentroid(metric='chebyshev')
y_pred = best_nc.fit(X_train, y_train).predict(X_valid)
accuracy_score(y_valid, y_pred)

Для центроидов оказалась более подходящий метрика Чебышева, однако результат всё равно хуже, чем у knn-метода. 

**Вывод:**
Самым эффективным из рассмотренных оказался knn-метод при 13 ближних соседях, основанный на манхэттенской метрике с весами, зависящими от расстояния, дающий почти 70% точности.